In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import *
from constants import *
from imports import *
from loaders import *
from train_utils import *
from rollout import RwEvaluator

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

('1.13.1', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'326j3j83'

In [3]:
wandb.init(id='372aji4n', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) 
m.train()
sum([torch.numel(p) for p in m.parameters()]) / 1000

30484.11

In [5]:
m = try_load_state_dict(m, torch.load(f"{BESPOKE_ROOT}/models/m.torch"))

In [6]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(m, wandb=wandb)

CPU times: user 24 µs, sys: 10 µs, total: 34 µs
Wall time: 37.4 µs


In [7]:
model_stem = "3.7"

In [8]:
%%time

rnn_only = True
rnn_is_first = False

if rnn_only: # m cnn backbone needs to be all loaded up and ready

    if rnn_is_first:
        freeze_model(m, True)
        freeze_model(m.rnn, False)
        m.hidden_init.requires_grad = True
        m.cell_init.requires_grad = True
    else:
        freeze_model(m.backbone, True)
        
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 21 #7 # these have to be multiple of Zloader's internal bptt, currently 3 for bs 32 or 7 for bs 16
    seqlen = bptt*10 #5 #15 
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    n_workers = 5 if seqlen > 100 else 4 if seqlen > 400 else 8 # TODO this doesn't work. dumbass. maxxing out ram here w higher bs. Was 8 w bs 16
    # fewer than five workers we start to have to wait too long for chunk. More is better here.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = 0 #.9 TODO temp disabled
    n_workers = 8
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
    
lr = 3e-4
m.use_rnn = rnn_only

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


...

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
CPU times: user 4.2 s, sys: 32.6 s, total: 36.8 s
Wall time: 2min 27s


Process Process-4:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process Process-2:
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 121, in keep_chunk_filled
    time.sleep(0.1)
KeyboardInterrupt
Process Process-3:
Process Process-6:
Traceback (most recent call last):
Process Process-5:
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call

In [9]:
opt = torch.optim.Adam(m.parameters(), lr=lr)
#opt = torch.optim.AdamW(m.parameters(), lr=3e-4, weight_decay=.001)

In [10]:
m.backbone.training, m.training

(False, True)

In [11]:
[(n,p.requires_grad) for (n,p) in m.named_parameters()]

[('hidden_init', True),
 ('cell_init', True),
 ('backbone.conv_stem.weight', False),
 ('backbone.bn1.weight', False),
 ('backbone.bn1.bias', False),
 ('backbone.blocks.0.0.conv_dw.weight', False),
 ('backbone.blocks.0.0.bn1.weight', False),
 ('backbone.blocks.0.0.bn1.bias', False),
 ('backbone.blocks.0.0.se.conv_reduce.weight', False),
 ('backbone.blocks.0.0.se.conv_reduce.bias', False),
 ('backbone.blocks.0.0.se.conv_expand.weight', False),
 ('backbone.blocks.0.0.se.conv_expand.bias', False),
 ('backbone.blocks.0.0.conv_pw.weight', False),
 ('backbone.blocks.0.0.bn2.weight', False),
 ('backbone.blocks.0.0.bn2.bias', False),
 ('backbone.blocks.0.1.conv_dw.weight', False),
 ('backbone.blocks.0.1.bn1.weight', False),
 ('backbone.blocks.0.1.bn1.bias', False),
 ('backbone.blocks.0.1.se.conv_reduce.weight', False),
 ('backbone.blocks.0.1.se.conv_reduce.bias', False),
 ('backbone.blocks.0.1.se.conv_expand.weight', False),
 ('backbone.blocks.0.1.se.conv_expand.bias', False),
 ('backbone.block

In [12]:
%%time
"""m.model_stem = model_stem
rw_evaluator.evaluate()"""

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 5.96 µs


'm.model_stem = model_stem\nrw_evaluator.evaluate()'

In [13]:
for n, mm in m.backbone.named_modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .99 #.4

In [14]:
trainer = Trainer(dataloader_trn, m, model_stem=model_stem, opt=opt, wandb=wandb, 
                  rw_evaluator=rw_evaluator, rnn_only=rnn_only)

In [15]:
trainer.reload_state()

Loading model from /home/beans/bespoke/models/m.torch. 
Last modified 3 min ago.
Loading opt from /home/beans/bespoke/models/opt.torch. 
Last modified 3 min ago.
Currently at epoch 31.


In [16]:
copy_cnn_params = False
if copy_cnn_params:
    m.copy_cnn_params_to_rnn() # when first trn rnn, has to be after reload_state.
    print("copied")
    
if rnn_only:
    trainer.loss_manager.update_emas = False
    trainer.loss_manager.loss_emas['te'] = .002 # manual. Bc cnn doesn't cache this.

In [17]:
trainer.loss_manager.update_emas

False

In [18]:
trainer.loss_manager.loss_weights

{'has_stop': 0.1,
 'stop_dist': 0.1,
 'has_lead': 0.1,
 'lead_dist': 0.1,
 'lead_speed': 0.02,
 'lane_width': 0.02,
 'dagger_shift': 0.05,
 'rd_is_lined': 0.02,
 'left_turn': 0.01,
 'right_turn': 0.01,
 'td': 0.03,
 'pitch': 0.1,
 'yaw': 0.1,
 'unc': 0.1,
 'te': 0.05,
 'wp_angles': 1,
 'wp_angles_i': 0.5,
 'wp_curvatures': 0.2,
 'wp_curvatures_i': 0.1,
 'wp_headings': 0.02,
 'wp_headings_i': 0.01,
 'wp_rolls': 0.05,
 'wp_rolls_i': 0.025,
 'wp_zs': 0.02,
 'wp_zs_i': 0.01}

In [19]:
trainer.loss_manager.loss_emas

{'has_stop': 0.009334587691325556,
 'stop_dist': 0.0018374660376339162,
 'has_lead': 0.009207439032678472,
 'lead_dist': 0.0012210935415941013,
 'lead_speed': 0.027568381558972448,
 'lane_width': 0.04535982400016158,
 'dagger_shift': 0.03166155214644604,
 'rd_is_lined': 0.00041711668115253773,
 'left_turn': 0.0014914766900982427,
 'right_turn': 0.0006639456156390354,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.426430307089145,
 'te': 0.002,
 'wp_angles': 0.09907790138498986,
 'wp_angles_i': 0.4239871905794027,
 'wp_curvatures': 0.7468527358573626,
 'wp_curvatures_i': 1.5990676294873805,
 'wp_headings': 0.11647753117860646,
 'wp_headings_i': 0.30532603654929497,
 'wp_rolls': 2.852011571212033,
 'wp_rolls_i': 0.008325651574648627,
 'wp_zs': 4.928500550682104,
 'wp_zs_i': 0.13389881089876574}

In [20]:
trainer.train()

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


177.75 7412.0 2400.0 27.203125 940.5 165.875 123.0625 246.125 5.0 600.0
wp_angles_i tensor(11.8438, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(14.1797, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(12.6797, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0135, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(1.4004, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(1.8604, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.2773, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(1.6465, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(13.1172, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(9.4688, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te tensor(0.0045, device='cuda:0

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning:

None of the inputs have requires_grad=True. Gradients will be None



156.75 642.0 6.03515625 18.09375 4.0390625 3.796875 22.546875 637.5 0.48681640625 547.5
2900.0 4388.0 380.0 375.5 446.5 227.375 74.5 368.5 2.236328125 356.0
50.96875 705.5 1683.0 19.078125 2011.0 8.015625 42.75 495.0 8.4296875 56.875
3058.0 1297.0 4336.0 100.5 432.0 4.53125 19.015625 133.375 8.6875 1764.0
wp_angles_i tensor(1.8633, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(8.1641, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(15.0547, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0267, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.2260, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0726, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1132, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.7788, device='cuda:0', dtype=torch.float

1169.0 3544.0 488.75 105.75 203.875 11.6796875 140.25 113.75 3.298828125 457.5
wp_angles_i tensor(2.3281, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(6.1719, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(19.1406, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0268, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.7344, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.1118, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1395, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.8394, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(7.8164, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(1.8623, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te tensor(0.0148, device='cu

...

233.375 2090.0 1588.0 39.0 172.25 21.0625 62.9375 79.9375 0.7578125 81.5
54.5 1108.0 7344.0 14.90625 791.0 4.78515625 7.26171875 444.25 1.986328125 63.71875
47.71875 3506.0 188.625 4.38671875 37.84375 6.7734375 11.265625 80.9375 0.39013671875 20.078125


 {'logistical/obs_consumed_per_second': 7.8203125, 'logistical/obs_generated_per_second': 24.3671875, 'logistical/slowest_runner_obs_per_sec': 3.0, 'logistical/data_consumption_ratio': 0.32410227, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 2.3268499, 'timing/model forward': 0.00642035, 'timing/calc losses': 0.06256382, 'timing/backwards': 0.20076941, 'timing/get worst': 1.241e-05, 'timing/logging': 0.0476395, 'timing/calc timing': 0.00932612, 'timing/trn update': 2.65358824, 'avg_unc': -2.81559753, 'wp_angles_i': 0.00821523, 'wp_headings_i': 0.01003657, 'wp_curvatures_i': 0.03289298, 'wp_rolls_i': 6.936e-05, 'wp_zs_i': 0.00322248, 'wp_angles': 0.00147517, 'wp_headings': 0.00166882, 'wp_curvatures': 0.0080

100.875 69.875 24.3125 5.62890625 5.22265625 3.77734375 7.625 167.0 0.16650390625 20.15625
46.0 847.0 2570.0 11.015625 51.65625 9.28125 10.1953125 439.0 0.313232421875 18.453125
2446.0 2698.0 4192.0 59.875 2810.0 8.640625 18.9375 169.0 1.3037109375 411.0
26.09375 126.25 10.2578125 12.6953125 12.6328125 14.1328125 19.046875 1187.0 1.044921875 1443.0
79.125 6324.0 392.25 9.765625 335.0 13.765625 55.28125 124.625 0.78955078125 857.0
wp_angles_i tensor(6.7500, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(4.2109, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(16.1250, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0015, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.4272, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0577, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1004,

42.5625 372.5 357.5 5.7265625 87.875 10.5390625 2.564453125 383.5 0.3505859375 42.125
217.0 480.0 829.0 43.84375 42.03125 62.25 26.46875 225.75 0.25146484375 268.5
wp_angles_i tensor(0.3694, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(1.2383, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(2.0918, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0031, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0795, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.1451, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1959, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(1.3936, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(5.7344, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(7.0977, device=



 {'logistical/obs_consumed_per_second': 8.0234375, 'logistical/obs_generated_per_second': 24.859375, 'logistical/slowest_runner_obs_per_sec': 2.59375, 'logistical/data_consumption_ratio': 0.32163605, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 2.35569596, 'timing/model forward': 0.00720434, 'timing/calc losses': 0.06122933, 'timing/backwards': 0.17610455, 'timing/get worst': 1.291e-05, 'timing/logging': 0.04631519, 'timing/calc timing': 0.0113439, 'timing/trn update': 2.65791232, 'avg_unc': -2.75920105, 'wp_angles_i': 0.00933074, 'wp_headings_i': 0.01100677, 'wp_curvatures_i': 0.04749931, 'wp_rolls_i': 6.18e-05, 'wp_zs_i': 0.00218875, 'wp_angles': 0.00115363, 'wp_headings': 0.00117454, 'wp_curvatures': 0.00582764, 'wp_rolls': 0.02420631, 'wp_zs': 0.04472759, 'te': 0.00253688, 'has_stop': 0.00861253, 'stop_dist': 0.00122814, 'has_lead': 0.01116902, 'lead_dist': 0.00134922, 'lead_speed': 0.02061783, 'dagger_shift': 0.02591721, 'lane_width': 0.03898174, 'rd

303.0 4192.0 1079.0 40.25 1269.0 34.78125 19.609375 108.375 0.9873046875 38.5625
352.75 3316.0 1275.0 10.25 306.25 4.125 119.5 151.625 20.234375 60.5625
63.625 372.5 404.75 25.390625 62.25 7.69140625 5.11328125 166.875 0.40283203125 19.84375


 {'logistical/obs_consumed_per_second': 7.3515625, 'logistical/obs_generated_per_second': 23.6875, 'logistical/slowest_runner_obs_per_sec': 3.0, 'logistical/data_consumption_ratio': 0.31146602, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 2.3777, 'timing/model forward': 0.00603857, 'timing/calc losses': 0.03288829, 'timing/backwards': 0.17482764, 'timing/get worst': 1.468e-05, 'timing/logging': 0.0402266, 'timing/calc timing': 0.01015602, 'timing/trn update': 2.64186244, 'avg_unc': -2.8469696, 'wp_angles_i': 0.00927238, 'wp_headings_i': 0.00988113, 'wp_curvatures_i': 0.03678914, 'wp_rolls_i': 0.00025951, 'wp_zs_i': 0.0020601, 'wp_angles': 0.00123032, 'wp_headings': 0.00130941, 'wp_curvatures': 0.00608763, 'wp_rolls': 

In [22]:
t = torch.randn(3,4,5)
t[0,0,0] = torch.inf

In [23]:
t

tensor([[[    inf, -0.7854, -1.3716,  0.8498, -0.0986],
         [ 0.3141, -1.5963, -0.3781, -1.1632,  0.0405],
         [-0.0689, -0.9696,  1.4334,  1.0205,  2.2483],
         [ 1.2481, -0.8255, -0.5951, -0.0256,  0.2744]],

        [[ 0.6463,  1.3897,  0.8417, -0.3971, -1.4846],
         [ 0.3036, -0.4071, -0.0882, -0.6457,  0.2220],
         [ 1.5495,  0.6217, -0.4204,  0.3186,  0.4908],
         [-2.7397, -2.3005,  1.9402, -0.5320, -1.1312]],

        [[ 0.2612,  0.7503, -0.6469,  0.5444, -0.6493],
         [-1.0282, -1.1678, -0.1762,  1.1714,  0.2500],
         [ 0.6518, -0.5325, -1.2205,  0.8746,  0.5583],
         [ 0.3824, -0.7756,  0.8413,  0.3293, -1.3277]]])

In [24]:
t.max()

tensor(inf)

In [20]:
AUX_TARGET_PROPS

['has_stop',
 'stop_dist',
 'has_lead',
 'lead_dist',
 'lead_speed',
 'dagger_shift',
 'lane_width',
 'rd_is_lined',
 'left_turn',
 'right_turn']

In [20]:
trainer.loss_manager.loss_emas

{'has_stop': 0.6581827152218972,
 'stop_dist': 0.1914074304465102,
 'has_lead': 0.6045025000545183,
 'lead_dist': 0.18079276826857715,
 'lead_speed': 0.8834656769419045,
 'lane_width': 4.096574170043749,
 'dagger_shift': 2.512469867257369,
 'rd_is_lined': 0.10581200184235365,
 'left_turn': 0.08856740587054233,
 'right_turn': 0.07401701428385941,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 117.34792707051348,
 'te': 0.002,
 'wp_angles': 0.0859911379087128,
 'wp_angles_i': 4.15318218310992,
 'wp_curvatures': 0.7827851002866839,
 'wp_curvatures_i': 17.866540970943753,
 'wp_headings': 0.12228421566478304,
 'wp_headings_i': 2.901144808827211,
 'wp_rolls': 4.211842307760256,
 'wp_rolls_i': 0.13459417300430104,
 'wp_zs': 5.043518468202303,
 'wp_zs_i': 1.1309414719989206}

In [20]:
m.hidden_init, m.cell_init

(Parameter containing:
 tensor([[[ 0.0015, -0.0488, -0.0292,  ..., -0.0560, -0.0283,  0.0664]]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[[-0.3869,  0.0330,  0.0708,  ..., -0.0204,  0.3181, -0.2530]]],
        device='cuda:0', requires_grad=True))

In [21]:
m.training, m.backbone.training 

(True, False)

In [22]:
set_trainer_should_stop(False)

In [28]:
{k:trainer.loss_manager.loss_emas["wp_angles"]/trainer.loss_manager.loss_emas[k] for k,v in trainer.loss_manager.loss_emas.items()}

{'has_stop': 29.734797363340526,
 'stop_dist': 87.87280383313023,
 'has_lead': 40.201898525296386,
 'lead_dist': 127.76564504863305,
 'lead_speed': 0.34846566491395553,
 'lane_width': 3.809900606793288,
 'dagger_shift': 4.134576796863424,
 'rd_is_lined': 16457.381372040072,
 'left_turn': 125.55969474655697,
 'right_turn': 283.74133223646584,
 'td': 0.015300805875045518,
 'pitch': 0.015300805875045518,
 'yaw': 0.015300805875045518,
 'unc': 0.11512757866420895,
 'te': 61.54656219577027,
 'wp_angles': 1.0,
 'wp_angles_i': 0.240031243936724,
 'wp_curvatures': 0.16382346075192508,
 'wp_curvatures_i': 0.003522151816451157,
 'wp_headings': 0.5183519273228973,
 'wp_headings_i': 0.005565440760627469,
 'wp_rolls': 0.027721036050055964,
 'wp_rolls_i': 8.663331955712781,
 'wp_zs': 0.00020285505128895626,
 'wp_zs_i': 0.0035411505624853203}

In [23]:
{'has_stop': 0.005145757574225003,
 'stop_dist': 0.001741244754645775,
 'has_lead': 0.0038059908701619494,
 'lead_dist': 0.0011975680840668381,
 'lead_speed': 0.4390907746627964,
 'lane_width': 0.04016064316156499,
 'dagger_shift': 0.0370069456362572,
 'rd_is_lined': 9.297229935401817e-06,
 'left_turn': 0.0012186080816722507,
 'right_turn': 0.0005392519219686348,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.329030459302299,
 'te': 0.002486053701322257,
 'wp_angles': 0.15300805875045517,
 'wp_angles_i': 0.6374505928519476,
 'wp_curvatures': 0.9339813604728601,
 'wp_curvatures_i': 1.44164213359285,
 'wp_headings': 0.2951818073498582,
 'wp_headings_i': 1.49253209788985,
 'wp_rolls': 5.519564942456264,
 'wp_rolls_i': 0.017661571729288113,
 'wp_zs': 754.2728553133405,
 'wp_zs_i': 43.208571917673005


import sys
sys.getsizeof(trainer.loss_manager.loss_emas)

1176

In [16]:
torch.save(trainer.model.state_dict(), f"{BESPOKE_ROOT}/models/m{trainer.model_stem}_e{trainer.current_epoch}.torch")

In [16]:
m.training

Error in callback <function _WandbInit._resume_backend at 0x7f62ef023160> (for pre_run_cell):


Exception: The wandb backend process has shutdown

True

Error in callback <function _WandbInit._pause_backend at 0x7f62e3b863a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
trainer.opt.param_groups[0]['lr'] = 3e-4

In [28]:
snr

0.15067877149872744



 {'logistical/obs_consumed_per_second': 38.2265625, 'logistical/obs_generated_per_second': 85.1328125, 'logistical/slowest_runner_obs_per_sec': 5.3125, 'logistical/data_consumption_ratio': 0.45004667, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00400971, 'timing/model forward': 0.19439457, 'timing/calc losses': 0.00539116, 'timing/backwards': 1.11790375, 'timing/get worst': 0.00264816, 'timing/logging': 0.01570235, 'timing/calc timing': 0.01359244, 'timing/trn update': 1.35364565, 'wp_angles': 0.01003864, 'wp_curvatures': 0.04547036, 'wp_headings': 0.01048879, 'wp_rolls': 0.52840137, 'wp_zs': 0.09037566, 'has_stop': 0.01459547, 'stop_dist': 0.00825622, 'has_lead': 0.01966284, 'lead_dist': 0.00662124, 'lead_speed': 16.11909485, 'dagger_shift': 0.14152849, 'lane_width': 0.12975717, 'rd_is_lined': 0.01347083, 'pitch': 4.037e-05, 'yaw': 4.98e-05, 'unc': 1.57641983, 'logistical/max_param': 32.69097137, 'logistical/lr': 0.0006, 'logistical/mins_since_slowest

In [9]:
%%time

rnn_only = True

if rnn_only: # m cnn backbone needs to be all loaded up and ready
    #freeze_model(m, True)
    #unfreeze_part_of_model(m, "rnn")
    freeze_model(m.backbone, True)
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 14 #35 #8 #7
    seqlen = bptt*5 # needs to be mult of 7
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = .9
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug)

Launching 3 loader workers


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated.

Got first chunk
CPU times: user 1.1 s, sys: 5.72 s, total: 6.83 s
Wall time: 45.2 s


Process Process-3:
Process Process-4:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 133, in make_chunks
    time.sleep(.1)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 1

In [25]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
